# MNIST 손글씨 - CNN 4
## - 2 Conv2d layers with pading
## - 1 MaxPooling 2D
## - 1 Dropout(0.25) layer
## - 1 Fully Connected Network layer
## - 1 Dropout(0.5) layer
## - 1 Output layer

In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.models import Sequential
#층을 담는 그릇(Sequential은 keras에 포함되어 있는데
# 딥러닝 모델을 한층 한층 쌓기 쉽게 해주는 함수)
from tensorflow.keras.layers import Dense
#층을 만드는 것(각 층이 가질 특성을 각각 다르게 지정 가능) 
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# seed값 생성
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

## 데이터 전처리

In [ ]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# 0~1 사이의 값으로 정규화
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
# 3차원 데이터를  4차원으로 (데이터는 2차원을 3차원으로)
X_train = X_train.reshape(-1, 28,28,1)
X_test = X_test.reshape(-1,28,28,1)
X_train.shape, X_test.shape

In [ ]:
# 원-핫 인코딩
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

## 모델 정의. 설정, 학습, 평가

In [ ]:
from tensorflow.keras.layers import Conv2D, Flatten, MaxPooling2D   # Conv2D - 2차원으로 / Flatten - 1차원으로
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping  # 모델 최고값 저장
from tensorflow.keras.layers import  MaxPool2D, Dropout

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), padding='same', input_shape=(28,28,1), activation='relu'),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    MaxPooling2D(pool_size=2),  # 1/2 줄어듦
    Dropout(0.25),
    Flatten(),               # 3차원을 1차원으로 변환
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# 모델 최적화를 위한 설정
modelpath = 'model/best_mnist_cnn_4.h5'
checkpoint = ModelCheckpoint(modelpath, verbose=1, save_best_only=True)
earlystopping = EarlyStopping(patience=5)

In [ ]:
# 모델 학습
history = model.fit(
    X_train, y_train,
    validation_split = 0.2,
    epochs=50, batch_size=200, verbose = 0,
    callbacks=[checkpoint, earlystopping])   # 트레인셋의 일부를 떼서 시험(split)

In [ ]:
from tensorflow.keras.models import load_model
best_model = load_model(modelpath)
best_model.evaluate(X_test, y_test)

## 훈련과정 시각화

In [ ]:
y_vloss = history.history['val_loss']
y_acc = history.history['accuracy']
x_len = np.arange(len(y_acc))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(12,8))

# 학습셋 정확도 라인
plt.plot(x_len, y_acc, 'bo-', markersize=2, label='accuracy')

# 태스트셋 오차 라인
plt.plot(x_len, y_vloss, 'ro-', markersize=2, label='val_loss')

plt.legend(loc='best')   #location  right, left, upper rigtht 등등 
plt.grid()   # 격자 
plt.xlabel('epoch')
plt.ylabel('score')

plt.show()